In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree

In [3]:
inspections = pd.read_csv("../Data/final_engineered_features.csv")

C:\Users\sambo\AppData\Local\Temp\ipykernel_70884\2659653593.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  inspections = pd.read_csv('C:\\Users\\sambo\\Downloads\\final_engineered_features.csv')


In [4]:
inspections.sample(5)

,FEI Number,Legal Name,City,State,Zip,Country/Area,Fiscal Year,Inspection ID,Posted Citations,Inspection End Date,...,FMD-145 Date,recalled_bool,classification_flag,inspection_year,inspection_month,inspection_dayofweek,inspection_quarter,week_of_year,inspection_season,days_since_last_inspection
297669,3014441790,1540 Vision Drive LLC,Platteville,Wisconsin,53818,United States,2019,1083307,No,2019-02-27,...,-,0,0,2019,2,2,1,9,Winter,-1.0
249643,3009517270,Daniel Hege,Waynesboro,Pennsylvania,17268,United States,2013,861445,No,2013-01-08,...,-,0,0,2013,1,1,1,2,Winter,-1.0
191274,3005065649,"Biomat USA, Inc.",Tacoma,Washington,98409,United States,2023,1190568,No,2022-11-03,...,2022-11-23 00:00:00,0,0,2022,11,3,4,44,Fall,1696.0
50106,2112482,Chalet Cheese Co-Op,Monroe,Wisconsin,53566,United States,2019,1102599,Yes,2019-09-23,...,-,0,1,2019,9,0,3,39,Fall,902.0
163521,3004268235,"Cooperative Producers, Inc.",Hastings,Nebraska,68901,United States,2018,1031507,No,2017-10-25,...,-,0,0,2017,10,2,4,43,Fall,1618.0


In [5]:
inspections = inspections[inspections['Product Type'] == 'Food/Cosmetics']

In [6]:
X = inspections.drop(['recalled_bool','classification_flag','Legal Name','City','Zip','FEI Number','Additional Details','FMD-145 Date','Inspection ID','Inspection End Date'], axis=1) 
y = inspections['recalled_bool']

In [7]:
X.sample(5)

,State,Country/Area,Fiscal Year,Posted Citations,Classification,Project Area,Product Type,inspection_year,inspection_month,inspection_dayofweek,inspection_quarter,week_of_year,inspection_season,days_since_last_inspection
188997,Massachusetts,United States,2011,No,Voluntary Action Indicated (VAI),Technical Assistance: Food and Cosmetics,Food/Cosmetics,2011,5,0,2,19,Spring,-1.0
182363,Pennsylvania,United States,2022,No,No Action Indicated (NAI),Foodborne Biological Hazards,Food/Cosmetics,2022,3,1,1,12,Spring,3288.0
303547,Wisconsin,United States,2023,Yes,Voluntary Action Indicated (VAI),"Food Composition, Standards, Labeling and Econ",Food/Cosmetics,2023,2,3,1,6,Winter,0.0
29991,Texas,United States,2020,No,No Action Indicated (NAI),"Food Composition, Standards, Labeling and Econ",Food/Cosmetics,2020,1,4,1,5,Winter,1051.0
162154,Iowa,United States,2009,No,No Action Indicated (NAI),"Food Composition, Standards, Labeling and Econ",Food/Cosmetics,2009,1,3,1,5,Winter,0.0


In [8]:
categorical_cols = ['Classification', 'Project Area', 'Product Type', 'State', 'Country/Area', 'inspection_season']
X = pd.get_dummies(X, columns = categorical_cols, drop_first = True, dtype = int)
X['Posted Citations'] = X['Posted Citations'].notnull().astype(int)

In [9]:
X.sample(5)

,Fiscal Year,Posted Citations,inspection_year,inspection_month,inspection_dayofweek,inspection_quarter,week_of_year,days_since_last_inspection,Classification_Official Action Indicated (OAI),Classification_Voluntary Action Indicated (VAI),...,Country/Area_United Kingdom,Country/Area_United States,Country/Area_Uruguay,Country/Area_Vanuatu,Country/Area_Venezuela,Country/Area_Vietnam,Country/Area_Western Samoa,inspection_season_Spring,inspection_season_Summer,inspection_season_Winter
104879,2017,1,2016,10,4,4,43,1003.0,0,1,...,0,1,0,0,0,0,0,0,0,0
12980,2022,1,2022,6,2,2,23,316.0,0,0,...,0,1,0,0,0,0,0,0,1,0
175495,2017,1,2017,2,4,1,8,1092.0,0,1,...,0,1,0,0,0,0,0,0,0,1
99806,2011,1,2011,8,2,3,31,383.0,1,0,...,0,1,0,0,0,0,0,0,1,0
243833,2019,1,2019,3,3,1,10,-1.0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [17]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, scoring='recall')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'bootstrap': [True, False],
                         'class_weight': [None, 'balanced'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='recall')

In [13]:
classifier = RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=10, # use the hyperparameters obtained via GridSearchCV
                       min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [14]:
scores = cross_val_score(
    classifier, X_train, y_train, cv=5, scoring='recall')
scores

array([0.88875355, 0.8956557 , 0.88595779, 0.87378247, 0.88839286])

In [16]:
avg_score = 0
for i in range(len(scores)):
    avg_score += scores[i] * (1/len(scores)) 
print(avg_score)

0.8865084747773543


In [17]:
classifier_final = RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=10,
                       min_samples_leaf=2, min_samples_split=5,
                       random_state=42)
classifier_final.fit(X_train, y_train)
y_pred = classifier_final.predict(X_test)